# Loading Llama 3 from pretrained weights

In [ ]:
from pprint import pprint

import torch

from gollem.models.llama3.config import get_llama3_model_config
from gollem.models.llama3.model import Llama3

cfg = get_llama3_model_config("llama-3.2-1B")
device = "cuda"

pprint(cfg)

model = Llama3.from_pretrained(cfg)
model.to(device)

In [9]:
tokenizer = cfg.get_tokenizer()
prompt = "On tuesday "
input_tokens = torch.tensor([tokenizer.encode(prompt)], device=device)
output_tokens = model.generate(input_tokens, 30)


In [ ]:
print(output_tokens.shape)
response = tokenizer.decode(output_tokens[0].tolist())
print(response)